In [15]:
import pandas as pd
import numpy as np
import requests
import csv

In [16]:
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text #requests.get(url).text will ping a website and return you HTML of the website.

We begin by reading the source code for a given web page and creating a BeautifulSoup (soup)object with the BeautifulSoup function. Beautiful Soup is a Python package for parsing HTML and XML documents. It creates a parse tree for parsed pages that can be used to extract data from HTML, which is useful for web scraping. Prettify() function in BeautifulSoup will enable us to view how the tags are nested in the document.

In [18]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(website_url,'html.parser')
#print(soup.prettify())

our first task is to find class "wikitable sortable" in the HTML script.

In [19]:
My_table = soup.find('table',{'class':'wikitable sortable'})# We can get this by inspecting the Wikipedia page

Now to extract all the links within 'tr tag', we will use find_all().

In [20]:
rows = My_table.find_all('tr')

In [21]:
header = [th.text.rstrip() for th in rows[0].find_all('th')]

with open('output.csv', 'w') as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(header)
    for row in rows[1:]:
        data = [td.text.rstrip() for td in row.find_all('td')]
        writer.writerow(data)


The first row ussually contains the header cells. We serch throught the first row in the rows list to get the text values of all th elements in that row.
we also ensure to remove the all trailing whitespaces in the text using the <b>rstrip</b> python string method.
The w mode is used to ensure the file is open for writing.
First we write the header row, then loop through the rest of the rows ignoring the first row to get the data contained within and write the data for all those rows to the file object.

In [22]:
df = pd.read_csv('output.csv')
df = df[df['Borough']!='Not assigned']#process only those rows where 'Borough' is not 'Not assigned'
df.head()

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [47]:
df1 = pd.DataFrame(df.groupby('Postal Code')['Postal Code'].count()>1)
df1.reset_index(drop=True, inplace=True)#Dropping the index column of a pandas.DataFrame removes the index column from the DataFrame and replaces it with the standard sequential indexing.
df1[df1['Postal Code']==True]

,Postal Code


From above code we found out that there are no repeating postal codes.

In [48]:
df.shape

(103, 3)